# Programa para interpolar os dados do BRAMS e MERGE em uma mesma grade de 0.01 x 0.01o
### Utiliza a biblioteca xesmf para interpolacao
#### BRAMS: interpolacao conservativa
#### MERGE: interpolacao bilinear

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import xarray as xr
import xesmf as xe

In [11]:
def interp(data,interp_type,filename):
    ds_out = xr.Dataset({'lat': (['lat'], np.linspace(-23.87, -23.22, 66)),
                     'lon': (['lon'], np.linspace(-47.08, -46.0, 109)),
                     'lon_b': np.linspace(-47.08-0.01/2, -46.0+0.01/2, 110),
                     'lat_b': np.linspace(-23.87-0.01/2, -23.22+0.01/2, 67),
                    }
                    )
    regridder = xe.Regridder(data, ds_out, interp_type)
    dr_out = regridder(data)
    ds_interp = xr.Dataset({
                "precip": dr_out})
    new_filename_ = './regrid_BRAMS_RMSP_1km_bilinear.nc'
    print ('saving to ',filename)
    ds_interp.to_netcdf(path=filename)
    return ds_interp

In [3]:
ds_brams = xr.open_dataset('file.nc') 
ds_brams

<xarray.Dataset> Size: 1MB
Dimensions:  (lon: 406, lat: 402)
Coordinates:
  * lon      (lon) float64 3kB -48.51 -48.5 -48.49 ... -44.56 -44.55 -44.54
  * lat      (lat) float64 3kB -25.38 -25.37 -25.36 ... -21.79 -21.78 -21.77
Data variables:
    precip   (lat, lon) float64 1MB ...

In [13]:
ds_merge = xr.open_dataset('file.nc') 
ds_merge

<xarray.Dataset> Size: 4kB
Dimensions:  (lon: 22, lat: 21)
Coordinates:
  * lon      (lon) float64 176B 312.5 312.6 312.7 312.8 ... 314.4 314.5 314.6
  * lat      (lat) float64 168B -24.55 -24.45 -24.35 ... -22.75 -22.65 -22.55
Data variables:
    precip   (lat, lon) float64 4kB ...

In [14]:
interp(ds_brams['precip'],'conservative', './regridBRAMS_conservative.nc')
interp(ds_merge['precip'],'bilinear', './regridMERGE_bilinear.nc')

saving to  ./regridBRAMS_conservative.nc
saving to  ./regridMERGE_bilinear.nc


<xarray.Dataset> Size: 59kB
Dimensions:  (lat: 66, lon: 109)
Coordinates:
  * lat      (lat) float64 528B -23.87 -23.86 -23.85 ... -23.24 -23.23 -23.22
  * lon      (lon) float64 872B -47.08 -47.07 -47.06 ... -46.02 -46.01 -46.0
Data variables:
    precip   (lat, lon) float64 58kB 8.645 8.739 8.834 ... 19.92 20.12 20.32